## Debugging mode
There are several configurations can be turned on to debug the simulator. As a result, you can get more console output from the simulation and visualizations such as coordinates and physics to help develop features or fix bugs. 

### Log messages




In [3]:
from metadrive.envs.scenario_env import ScenarioEnv
import numpy as np
import os
render = not os.getenv('TEST_DOC')

# create real-world environment with debug mode turned on
env = ScenarioEnv(dict(use_render=False, debug=True))  
env.reset()
env.logger.debug("debug message")
try:
    for i in range(100):
        env.step([0,1])
finally:
    env.close()

[INFO] MetaDrive version: 0.4.1.2
[INFO] Sensors: [lidar: Lidar(50,), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Assets version: 0.4.1.2
[WARNING] show_dest_mark and show_line_to_dest are not supported in TrajectoryNavigation (trajectory_navigation.py:35)



It is important to making sure that physics world is accurate. For example, objects need bounding box to collide with each other; Physics bodies should be correctly configured for lane lines, sidewalks and crosswalks so that the contact between these road components and objects can be detected. To visualize the physics bodies, Let's first turn on `debug` mode. 